In [1]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision

In [3]:
# TODO

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(28, 300) #input dim = 28, hidden = 300
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 50)
        self.fc4 = nn.Linear(50, 1)
        

    def forward(self, x): # tanh, sigmoid?
        a = self.fc2(self.fc1(x))
        a = self.fc3(a)
        a = self.fc4(a) # TODO : relu
        
        return a

In [4]:
import set_data as sd

# load train set, test set

X_train, Y_train, X_test, Y_test = sd.set_data("training_data")

x_train = Variable(X_train)
y_train = Variable(Y_train)


In [8]:
# train
# hyperparameter

epochs = 500
learningrate = 0.01
# batch_size
# hidden_layer_size = 300

model = RegressionModel()
criterion = nn.MSELoss(size_average = False) 
optimizer = torch.optim.SGD(model.parameters(), lr=learningrate) # or Adam

print('\nStart Training...\n')

for epoch in range(epochs):
    # forward pass
    pred_y = model(x_train)

    # compute and print loss
    loss = criterion(pred_y, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    tmp_loss = []
    tmp_loss.append(loss.data[0])
    
    mbl = np.mean(np.sqrt(tmp_loss)).round(3)
    
    if e % 10 == 0:
        print ('epoch [{}/{}], loss {}'.format(epoch, epochs, mbl)) # or loss item
    


Start Training...



RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm

In [ ]:
torch.save(model.state_dict(), "model.pt")

In [ ]:
# test

print ('\nStart Testing...\n')

model.eval()

# calculate R squared
# TODO

In [ ]:
# visualize : TODO
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['training', 'validation'], loc = 'upper left')
plt.show()